# 1- EXPLORATORY DATA ANALYSE

## INTRODUCTION

## Objectif : 
- Comprendre au maximum les données dont on dispose pour définir une stratégie de modélisation

- Dévolopper une première stratégie de modélisation

#### ANALYSE DE LA FORME : 

- **Identification de la target** : output

- **Nombre de lignes et de colonnes** : 303 lignes et 14 colonnes

- **Types de variables** : qualitatives : 9, quantitatives : 5

- **Identification des valeurs manquantes** : aucune NaN

#### ANALYSE DE LA FORME

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
data.head()

In [ ]:
#Copie du dataframe
df = data.copy()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
cat_col = ['sex','cp','fbs', 'restecg','exng', 'slp', 'caa', 'thall']
num_col = ['age','trtbps','chol', 'thalachh','oldpeak' ]

In [ ]:
for col in cat_col : 
    df[[col]] = df[[col]].astype(int)
    
for col in num_col : 
    df[[col]] = df[[col]].astype(float)

In [ ]:
print('Nombre de variables numériques :' , df.select_dtypes(float).shape[1])
print('Nombre de variables catégorielles :' ,df.select_dtypes(int).shape[1])

In [ ]:
# Analyse des valeurs manquantes

plt.figure(figsize = (10,7))
    
sns.heatmap(df.isna(), cbar = False)

In [ ]:
df.isna().sum()

#### ANALYSE DE FOND : 

- **Visualisation de la target (Histogramme si c’est une valeur continue / Boxplot si c’est une valeur discrète** : 
    - Dataset équilibré avec 54% de cas positifs

- **Signification des différentes variables** :
    - Variables continues : non-standardisées, oldpeak n'a pas une distribution normale
    - Variables age : age varie de 29 et 77 ans, on pourra créer une variable pour les catégories d'age plus tard
    - Variables qualitatives :
      
       binaire : 
    - fbs (gycémie a jeun), 
    - exng(angine du coeur)
    
      catégorielles : 
    - thall (test majorité 3 et 1 : défaut réversible ou ordinaire)
    - caa (vaissaux pricipaux colorés : maj 0 - normal puis 1 - ST anormal puis 2- ventricule     hypertrophié)
    - slope (majorité de 2 et 1 : plate ou descendante)
    - restecg éléctrocardiogramme au repos (moitié 0 et 1 et tres peu de 2)
    - cp douleur cardiaque (0- asymptomatique et 2- angine atypique peu de 1- angine typique     et 3- pas de douleurs )
    

- **Relations features – target (Histogramme / Boxplot)** :
    - target / catégorie : 
    
     - sex : Les hommes semblent plus touchés par les arrêts cardiaques
     - cp : Les douleurs cardiaques « typical angina » et « atipical angina” semblent causer plus d’arrêts cardiaques
     - restecg : Les résultats d’éléctrocardiogrammes qui montrent une anomalie de l’onde  ST-T semblent mener à plus d’arrêts cardiaques 
     - exng : Pas d’angore pendant l’exercice semble augmenter les chances d’arrêt cardiaque 
     - slp : Une pente descendante du segment ST lors l'effort de pointe semble augmenter les  chances d’arrêt cardiaque 
     - caa : 0 vaisseaux colorés par la fluotherapie semble augmenter les chances d’arrêt cardiaque 

    - target / age : 
     - l'age ne semble pas corrélé aux problèmes cardiaques qui augmentent autour de 40-44 ans puis 50-54 ans puis dimimnuent apres 56 ans
   
    - target / variables numériques :les variables qui semblent avoir le plus d'impact sont  : 
     - thalach :les personnes qui font des arrets cardiaques ont un taux old peak près de 0
     - oldpeak : les personnes qui font des arrets cardiaquesune fréquence maximale atteinte différente (plus élevée) 

In [ ]:
# VARIABLE CIBLE 
df['output'].value_counts(normalize = True)

In [ ]:
# VARIABLES NUMÉRIQUES

for col in num_col : 
    plt.figure()
    sns.distplot(df[col], bins = 20)

In [ ]:
# VARIABLE AGE

sns.boxplot(df['age'], data = df)

In [ ]:
plt.figure(figsize = (12,8))
sns.distplot(df['age'], bins = 100)

In [ ]:
print(df['age'].min())
print(df['age'].max())

In [ ]:
# VARIABLES CATÉGORIELLES

for col in cat_col : 
    plt.figure()
    df[col].value_counts().plot.pie()

#### Relation Target / Variables

##### Création de sous-ensembles positifs et négatifs

In [ ]:
df_positive = df[df['output'] == 1]
df_negative = df[df['output'] == 0]

In [ ]:
plt.figure(figsize = (12,8))
ax = sns.countplot(y="age", hue="output", data=df)

In [ ]:
for col in num_col : 
    plt.figure()
    sns.distplot(df_negative[col], label = 'negative')
    sns.distplot(df_positive[col], label = 'positive')
    plt.legend()

In [ ]:
ncount = len(df)

ax = sns.countplot(x="sex", hue="output", data=df, palette = "cubehelix")
             
for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom') # set the alignment of the text


In [ ]:
for i,j in zip(cat_col, range(len(cat_col))[-7:]) : 
    ncount = len(df)
    plt.figure(figsize = (10,30))
    plt.subplot(9,1,j)
    ax = sns.countplot(x= i, hue="output", data=df, palette = "cubehelix")
    
    for p in ax.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom') # set the alignment of the text




In [ ]:
sns.countplot(x= 'thall', hue="output", data=df, palette = "cubehelix")

#### ANALYSE PLUS DETAILLEE

- **Relations variables / Variables** : 

- L’age est corrélé avec la thalach (fréquence cardiaque maximale atteinte)
- La douleur au cœur cp est corrélé avec exng (angine du cœur)
- L’angine du cœur(exng) est corrélé avec la fréquence cardiaque maximale atteinte (thalach)

- Slope (slp) La pente du segment ST de l'effort de pointe est corrélé avec la fréquence cardiaque maximale atteinte (thalach)

- Slope (slp) La pente du segment ST de l'effort de pointe est corrélé avec oldpeak la dépression ST induite par l'exercice par rapport au repos

- Toutes les variables semblent corrélées avec la variables cible sauf : 
 •	chol(cholesterol), 
 •	fbs (glycémie à jeun), 
 •	resecg(résultats électrocardiographiques au repos), 
 •	trtbps (tension artérielle au repos (en mm Hg))

  


##### Hypothèses nulles (H0) : 
- L’age est corrélé avec output : Anova
- L’angine du cœur (exng) est corrélé avec (thalach ) : Anova
- L’angine du cœur (exng) est indépendante de (cp) et (slp) : khi 2
- Output est corrélé avec cp, exng, caa: khi2
- Output est corrélé avec thallach, oldpeak,  : Anova


In [ ]:
plt.figure(figsize = (12,8))
sns.heatmap(df.corr(),cmap='coolwarm',  annot = True)

In [ ]:
corr = df.corr()
corr[(df.corr() >= 0.4) | (df.corr() <= - 0.39)]

In [ ]:
num_col

In [ ]:
cat_col

In [ ]:
from scipy.stats import chi2_contingency

def chitest_target (col): 
    table = pd.crosstab(df[col], df['output'])
    result_test = chi2_contingency(table)

    print(col)
    
    print('statistique du test',result_test[0] )
    print('p_value', "%.16f" % float(result_test[1]))
    print('degré de liberté',result_test[2] )
    

In [ ]:
test_col = ['cp','exng','caa' ]

for col in test_col : 
    print(f'{col}{chitest_target(col)}')

In [ ]:
def chitest_exng (col): 
    table = pd.crosstab(df[col], df['exng'])
    result_test = chi2_contingency(table)

    print(col)
    
    print('statistique du test',result_test[0] )
    print('p_value', "%.16f" % float(result_test[1]))
    print('degré de liberté',result_test[2] )

In [ ]:


test_col = ['cp','slp' ]

for col in test_col : 
    print(f'{col}{chitest_exng(col)}')

In [ ]:
sns.pairplot(df[num_col])

In [ ]:
#Relation Olpeak / Slt

sns.catplot(x = 'slp', y =df['oldpeak'], data = df, hue = 'output')

In [ ]:
# Realtion thalach /rest_ecg :
    
sns.catplot(x = 'restecg', y =df['thalachh'], data = df, hue = 'output')

In [ ]:
# Realtion chol /fbs :

sns.catplot(x = 'fbs', y =df['chol'], data = df)

In [ ]:
#Relation sexe / thalachh

plt.figure(figsize=(10,6))

df[df['sex']==1]['thalachh'].hist(alpha=0.5,color='blue',
                                              bins=30,label='sex=1')
df[df['sex']==0]['thalachh'].hist(alpha=0.5,color='red',
                                              bins=30,label='sex=0')
plt.legend()
plt.xlabel('Thalachh')

In [ ]:
figure = plt.figure(figsize=(20,8))

sns.boxplot(x= 'sex',y='thalachh',hue = 'restecg',data=df)

In [ ]:
#Relation age/ thall

figure = plt.figure(figsize=(15,6))
sns.barplot(x= 'age', y ='thall',data=df)

# 2- PRE TRAITEMENT DES DONNÉES

In [ ]:
cat_col 

In [ ]:
num_col

In [ ]:
del num_col[0]

In [ ]:
key_col = ['age', 'output']

In [ ]:
df = data.copy()

df = df[cat_col+num_col + key_col]

#### TRAIN TEST SET

In [ ]:
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(df, test_size = 0.2, random_state = 0)

In [ ]:
trainset['output'].value_counts(normalize = True)

In [ ]:
testset['output'].value_counts(normalize = True)

#### FONCTIONS DE PREPROCESSING

In [ ]:
df['cp'] = df['cp'].map({ 0 :'asympto',
                             1 :'typ angina',
                             2 :'atyp angina',
                        3 :'non angina pain'})
    
    
df['restecg'] = df['restecg'].map({ 0 :'normal',
                             1 :'ST wave anormal',
                             2 :'hypertrophy'})

    
df['slp'] = df['slp'].map({ 2 :'croissant',
                             1 :'plat',
                             0 :'descendant'})

df['thall'] = df['restecg'].map({ 0 :'result0',
                             1 :'result1',
                             2 :'result2',
                        3 :'result3'})

df_dumm = pd.get_dummies(df[['cp', 'restecg','slp', 'thall']])

df = df.join(df_dumm)

df = df.drop(['cp', 'restecg','slp', 'thall'], axis = 1)

In [ ]:
df['age_cat'] = pd.cut(df['age'], bins=[x for x in range(0,100, 10)],labels=[x for x in range(10,100, 10)], right=True)

In [ ]:
df['ST_problem'] = 0
df.loc[(df['oldpeak'] <= 0.9), 'ST_problem'] = 1
df.loc[(df['slp_croissant'] == 1), 'ST_problem'] = 1
df.loc[(df['slp_descendant'] == 1), 'ST_problem'] = 1
df.loc[(df['restecg_ST wave anormal'] == 1),'ST_problem'] = 1
df.loc[(df['restecg_hypertrophy'] == 1),'ST_problem'] = 1



In [ ]:
df['angor'] = 0
df.loc[(df['exng'] == 1), 'angor'] = 1
df.loc[(df['cp_typ angina'] == 1), 'angor'] = 1

In [ ]:
df

In [ ]:
def preprocessing (df) : 
    
    #df = encodage (df) 
    #df = feature_engineering (df)
    
    X = df.drop('output', axis = 1)
    y = df['output']
    
    print(y.value_counts(normalize = True))
    
    return X,y
    

In [ ]:
trainset, testset = train_test_split(df, test_size = 0.2, random_state = 0)

In [ ]:
X_train, y_train = preprocessing(trainset)
X_test, y_test = preprocessing (testset)
#X_test['restecg_hypertrophy'] = 0

In [ ]:
y_test.value_counts()

#### 1iere Modélisation

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.ensemble import  RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA


In [ ]:
#model = DecisionTreeClassifier(random_state = 0)

model = make_pipeline(PolynomialFeatures(2),SelectKBest(chi2,k=17), 
                      RandomForestClassifier(random_state = 0))

#model = make_pipeline(PolynomialFeatures(2),PCA(n_components = 17),
                      #RandomForestClassifier(random_state = 0))

In [ ]:
def evaluation(model) : 
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    
    N,train_score, val_score = learning_curve(model,X_train,y_train, 
                                              cv = 4, scoring = 'f1', 
                                             train_sizes = np.linspace(0.1,1,10))
    
    plt.figure()
    plt.plot(train_score.mean(axis = 1), label = 'train score')
    plt.plot(val_score.mean(axis = 1), label = 'val score')
    plt.legend()

In [ ]:
evaluation(model)

In [ ]:
# pd.DataFrame(data = model.feature_importances_, index = X_train.columns).plot.bar()

In [ ]:
df.columns.to_list()

In [ ]:
u = np.arange(1,21,1)

for i in u :
    model = make_pipeline(SelectKBest(f_classif,k=i), 
                      RandomForestClassifier(random_state = 0))
    
    print(i)
    evaluation(model)

# 3- MODELISATION

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
preprocessor = make_pipeline(PolynomialFeatures(2),SelectKBest(chi2, k = 17))

SGDClassifier = make_pipeline(preprocessor,StandardScaler(),SGDClassifier(random_state = 0))

SVC = make_pipeline(preprocessor, StandardScaler(), SVC(random_state = 0))

AdaBoost = make_pipeline(preprocessor, AdaBoostClassifier(random_state = 0))

KNN = make_pipeline(preprocessor,StandardScaler(), KNeighborsClassifier())


In [ ]:
dict_of_model = {'SGD' : SGDClassifier, 
                 'SVC' : SVC , 
                 'AdaBoost' : AdaBoost, 
                 'KNN' : KNeighborsClassifier }


In [ ]:
for name, model in dict_of_model.items():
    print(name)
    evaluation(model)

In [ ]:
model = make_pipeline(preprocessor,StandardScaler(), KNeighborsClassifier())
evaluation(model)

# 3.1- MODELISATION AVEC SVC

In [ ]:
evaluation(SVC)

# SVM : OPTIMISATION GRID SEARCH CV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
hyper_params = {'svc__gamma' : [1e-3, 1e-4, 0.01,1,10,100,1000],
                'svc__C' : [1,10,100,1000],
                'svc__kernel' :['linear', 'poly', 'rbf', 'sigmoid']
}

In [ ]:
grid = GridSearchCV(SVC, hyper_params, scoring = 'f1', cv=4)

grid.fit(X_train,y_train)

print(grid.best_params_)

In [ ]:
y_pred = grid.predict(X_test)
evaluation(grid.best_estimator_)

# SVM : OPTIMISATION AVEC RANDOMIZEDSEARCHCV

In [ ]:
hyper_params = {'svc__gamma' : [1e-4, 0.01,1],
                'svc__C' : [1,10],
                'svc__kernel' :['linear', 'poly'],
               'pipeline__polynomialfeatures__degree' : [2,3,4],
               'pipeline__selectkbest__k' : range(15,22)}

In [ ]:
grid = RandomizedSearchCV(SVC, hyper_params, scoring = 'f1', cv=4)

grid.fit(X_train,y_train)

print(grid.best_params_)

In [ ]:
y_pred = grid.predict(X_test)

evaluation(grid.best_estimator_)

# 3.2- MODELISATION AVEC KNN

In [ ]:
preprocessor = make_pipeline(PolynomialFeatures(2),SelectKBest(chi2, k = 17))
KNN = make_pipeline(preprocessor, StandardScaler(), KNeighborsClassifier())

evaluation(KNN)

# KNN : OPTIMISATION GRID SEARCH CV

In [ ]:
hyper_params = { 'kneighborsclassifier__n_neighbors' : range(2,10),
                'kneighborsclassifier__weights' : ['uniform', 'distance'],
                'kneighborsclassifier__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}



In [ ]:
grid = GridSearchCV(KNN, hyper_params, scoring = 'recall', cv = 4)
grid.fit(X_train, y_train)

print(grid.best_params_)

In [ ]:
y_pred = grid.predict(X_test)

evaluation(grid.best_estimator_)

# KNN : OPTIMISATION RANDOMIZED SEARCH CV

In [ ]:
hyper_params = {'kneighborsclassifier__n_neighbors' : range(4,10),
                'kneighborsclassifier__weights' : ['uniform', 'distance'],
                'kneighborsclassifier__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
               'pipeline__polynomialfeatures__degree' : [2,3,4],
               'pipeline__selectkbest__k' : range(15,22)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

grid = RandomizedSearchCV(KNN, hyper_params, scoring = 'f1', cv = 4)
grid.fit(X_train, y_train)

print(grid.best_params_)

In [ ]:
y_test = grid.predict(X_test)

evaluation(grid.best_estimator_)